<a href="https://colab.research.google.com/github/QuantAnalyticsTorch/quant_analytics_torch/blob/main/examples/RandomPerformance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import tensorflow as tf

In [19]:
from scipy.special import erf
import numpy as np
import time

In [32]:
x = np.random.rand(20000000)

In [39]:
#@title Pricing time a CPU. Note TensorFlow does automatic multithreading.


# First run (includes graph optimization time)
with tf.device("/cpu:0"):
    #y = erf(x)
    y = tf.math.erf(x)

# Second run (excludes graph optimization time)
time_start = time.time()
with tf.device("/cpu:0"):
    #y = erf(x-0.5)
    y = tf.math.erf(x-0.5)
time_end = time.time()

time_price_cpu = time_end - time_start
print("Time (seconds) to price a European Call Option on a CPU: ", time_price_cpu)

Time (seconds) to price a European Call Option on a CPU:  0.14335966110229492


In [32]:
class BrownianBridge():
    def __init__(self, number_time_steps):
        super(BrownianBridge, self).__init__()

        self.number_time_steps = number_time_steps

        self.left_index = np.zeros(number_time_steps, dtype=int)
        self.right_index = np.zeros(number_time_steps, dtype=int)
        self.bridge_index = np.zeros(number_time_steps, dtype=int)
        self.left_weight = np.zeros(number_time_steps)
        self.right_weight = np.zeros(number_time_steps)
        self.std_dev = np.zeros(number_time_steps)

        self._map = np.zeros(number_time_steps, dtype=int)

        self._map[-1] = 1
        self.bridge_index[0] = number_time_steps - 1
        self.std_dev[0] = tf.math.sqrt(1.0 * number_time_steps)
        self.left_weight[0] = 0
        self.right_weight[0] = 0

        j=0
        for i in range(1,number_time_steps):
            while self._map[j] == True:
                j = j + 1
            k = j
            while self._map[k] == False:
                k = k + 1
            l = j+((k-1-j)>>1)
            self._map[l]=i
            self.bridge_index[i]=l
            self.left_index[i]=j
            self.right_index[i]=k
            self.left_weight[i]=(k-l)/(k+1-j)
            self.right_weight[i]=(1+l-j)/(k+1-j)
            self.std_dev[i]=np.sqrt(((1+l-j)*(k-l))/(k+1-j))
            j=k+1
            if j>=number_time_steps:
                j=0
    
    def buildPath(self, path, z):
        path[-1] = self.std_dev[0]*z[0];
        for i in range(1,self.number_time_steps):
            j = self.left_index[i]
            k = self.right_index[i]
            l = self.bridge_index[i]
            if j > 0:
                path[l] = self.left_weight[i] * path[j-1] + self.right_weight[i] * path[k] + self.std_dev[i] * z[i]
            else:
                path[l] = self.right_weight[i] * path[k] + self.std_dev[i] * z[i]
        return path



In [33]:
bridge = BrownianBridge(8)

In [34]:
path = np.zeros(shape=(8,10))

z = np.random.randn(8,10)

w = bridge.buildPath(path,z)